In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import math

seed = 42
np.random.seed(seed)
random.seed(seed)

################## --- Functions and Parameters --- ########################

def arrival_rates(N):
    rates = np.random.uniform(0.1, 0.5, size=(1, N))
    return rates

def arrival_function(R, N, rates):
    return np.random.poisson(lam=rates, size=(R, N))
 
def service_rate(Q):
    return mu_0_values / (1 + (alpha * Q))  

def compute_average(Q):
    return np.mean(Q, axis=0, keepdims=True)

def compute_chi(Q, Q_avg):
    return (Q > Q_avg).astype(int)

n_grid = 10 # 10x10 grid => 100 nodes.
grid_nodes = [(i, j) for i in range(1, n_grid+1) for j in range(1, n_grid+1)]
total_nodes = len(grid_nodes) 
sinks = [] 

num_nodes = total_nodes  
beta = 0.7
T = 1000
runs = 100
min_mu_0 = 1  # Minimum service
max_mu_0 =  5 # Maximum service
alpha = 0.01
mu_0_values = np.random.uniform(min_mu_0, max_mu_0, size=num_nodes)


################# ------ Building the Network -------- ######################

def build_grid(n, sinks):
    grid_nodes = [(i, j) for i in range(1, n+1) for j in range(1, n+1)]
    G = nx.DiGraph()
    G.add_nodes_from(grid_nodes)

    # Add right and down edges (original grid structure)
    for i in range(1, n+1):
        for j in range(1, n+1):
            if j < n:
                G.add_edge((i, j), (i, j+1))
            if i < n:
                G.add_edge((i, j), (i+1, j))
    return G

# Build the grid graph.
G_grid = build_grid(n_grid, sinks)

# Create a mapping from grid node to an index.
sorted_nodes = sorted(G_grid.nodes())
node_to_idx = {node: idx for idx, node in enumerate(sorted_nodes)}

# Build dictionary of out-neighbors by index.
out_neighbors = {}
for node in sorted_nodes:
    idx = node_to_idx[node]
    out_neighbors[idx] = [node_to_idx[s] for s in G_grid.successors(node)]

# --- Plot the grid network ---
pos = {node: (node[1], -node[0]) for node in G_grid.nodes()}
node_colors = ['red' if node == sorted_nodes[-1] else 'skyblue' for node in sorted_nodes]
plt.figure(figsize=(10, 8), constrained_layout=True)
nx.draw(G_grid, pos, with_labels=True, labels={node: str(node) for node in G_grid.nodes()},
        node_color=node_colors, node_size=300, edge_color='gray', arrows=True)
plt.show()


################# --- Simulation --- ########################

# We have total external arrivals (A), departures (D), and incoming data (F) over time.
A_cum = np.zeros((runs, num_nodes))  # cumulative external arrivals
D_cum = np.zeros((runs, num_nodes))  # cumulative departures (transmissions)
F_cum = np.zeros((runs, num_nodes))  # cumulative incoming data (from upstream)

# For record keeping
chi_history = np.zeros((T, runs, num_nodes))
Q_history = np.zeros((T, runs, num_nodes))

# For initial arrivals rates
rates = arrival_rates(num_nodes)

# Initialize queue at time 0: no queueing initially.
Q = np.zeros((runs, num_nodes))
Q_history[0] = Q.copy()
A = arrival_function(runs, num_nodes, rates)
A_cum += A

# Assume no received data at t=1, and set sink to zero.
Q = (1-beta)*A_cum 
Q[:, -1] = 0
Q_history[1] = Q.copy()

# --- Simulation Loop ---
for t in range(2, T):
    print(t)
    # 1) External arrivals at time t
    A_new = arrival_function(runs, num_nodes, rates)
    
    # 2) Compute control variable chi: transmit if Q > average queue length, else no.
    Q_avg = compute_average(Q)
    chi = (Q > Q_avg).astype(int)
    chi_history[t] = chi.copy()
    
    # 3) Compute instantaneous service rate based on current Q.
    sr = service_rate(Q)  
    
    # 4) Compute outgoing transmissions for nodes that are scheduled to transmit.
    transmitted = np.zeros((runs, num_nodes))
    received = np.zeros((runs, num_nodes)) 
    
    for r in range(runs):
        for i in range(num_nodes):
            if chi[r, i] == 1 and out_neighbors[i]:
                # Sample transmitted data from Poisson with rate sr[r,i]
                T_i = np.random.poisson(lam=sr[r, i])
                # Ensure it cannot transmit more than what is in the queue:
                T_i = min(T_i, Q[r, i])
            else:
                T_i = 0
            transmitted[r, i] = T_i
            received[r, i] = sr[r, i]

    # 5) Update cumulative quantities:
    A_cum += (1-beta) * A_new
    F_cum += (1-beta) *received
    D_cum += transmitted

    A_cum[:, -1] = 0
    F_cum[:, -1] = 0

    A_cum =  A_cum
    F_cum =  F_cum 
    
    # 6) Compute new queue length using:
    # Q_i(t) = (1 - beta)[A_i(t) + F_i(t)] - D_i(t)
    Q =  (A_cum + F_cum) - D_cum
      
    # 8) Store the current state.
    Q_history[t] = Q.copy()

############### ---- Plotting the Queues ------ ###################

plt.rcParams.update({
    'font.size': 25,
    'text.usetex': False,  # Disable LaTeX rendering to avoid extra requirements
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'Computer Modern Roman', 'DejaVu Serif']
})

# Generate the time axis
time_axis = np.arange(T)
plt.figure(figsize=(12, 8), constrained_layout=True)

# Plot the average queue trajectory over multiple runs for each node
for i in range(0, num_nodes):
    plt.plot(
        time_axis,
        np.mean(Q_history[:, :, i], axis=1),
        lw=1,
        label=f"Node {sorted_nodes[i]}"
    )

plt.xlabel("Time Step", fontsize=25)
plt.ylabel("Average Queue Length", fontsize=25)
plt.grid(True)
plt.show()